In [1]:
import argparse
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
import ray
from torch import nn

from typing import *
from ray import air
from ray import tune
from baselines.train.configs import get_experiment_config
from ray.rllib.algorithms import ppo
from ray.tune import registry
from ray.air.integrations.wandb import WandbLoggerCallback
from baselines.train import make_envs

2024-05-24 15:32:19,539	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-24 15:32:26,421	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-24 15:32:32.428983: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-05-24 15:32:32.429072: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-05-24 15:32:32.429081: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvid

In [2]:
ckp = "results/torch/pd_matrix/PPO_meltingpot_397b4_00000_0_2024-05-21_13-58-39/checkpoint_007270"

registry.register_env("meltingpot", make_envs.env_creator)


In [8]:

def get_cli_args():

    parser = argparse.ArgumentParser(
        description="Training Script for Multi-Agent RL in Meltingpot"
    )

    parser.add_argument(
        "--num_workers",
        type=int,
        default=2,
        help="Number of workers to use for sample collection. Setting it zero will use same worker for collection and model training.",
    )
    parser.add_argument(
        "--num_gpus",
        type=int,
        default=0,
        help="Number of GPUs to run on (can be a fraction)",
    )
    parser.add_argument(
        "--local",
        action="store_true",
        help="If enabled, init ray in local mode.",
    )
    parser.add_argument(
        "--no-tune",
        action="store_true",
        help="If enabled, no hyper-parameter tuning.",
    )
    parser.add_argument(
        "--algo",
        choices=["ppo"],
        default="ppo",
        help="Algorithm to train agents.",
    )
    parser.add_argument(
        "--framework",
        choices=["tf", "torch"],
        default="torch",
        help="The DL framework specifier (tf2 eager is not supported).",
    )
    parser.add_argument(
        "--exp",
        type=str,
        choices=["pd_matrix", "clean_up"],
        default="pd_matrix",
        help="Name of the substrate to run",
    )
    parser.add_argument(
        "--seed",
        type=int,
        default=123,
        help="Seed to run",
    )
    parser.add_argument(
        "--results_dir",
        type=str,
        default="./results",
        help="Name of the wandb group",
    )
    parser.add_argument(
        "--logging",
        choices=["DEBUG", "INFO", "WARN", "ERROR"],
        default="INFO",
        help="The level of training and data flow messages to print.",
    )

    parser.add_argument(
        "--wandb",
        type=bool,
        default=False,
        help="Whether to use WanDB logging.",
    )

    parser.add_argument(
        "--downsample",
        type=bool,
        default=True,
        help="Whether to downsample substrates in MeltingPot. Defaults to 8.",
    )

    parser.add_argument(
        "--as-test",
        action="store_true",
        help="Whether this script should be run as a test.",
    )

    parser.add_argument(
        "--continue_training",
        type=str,
        default=None,
        help="Path to the checkpoint to continue training from.",
    )

    parser.add_argument(
        "--param_sharing",
        action="store_true",
        help="Whether to share parameters across agents.",
        
    )

    args = parser.parse_args(["--num_gpus","1"])
    print("Running trails with the following arguments: ", args)
    return args

args = get_cli_args()

Running trails with the following arguments:  Namespace(num_workers=2, num_gpus=1, local=False, no_tune=False, algo='ppo', framework='torch', exp='pd_matrix', seed=123, results_dir='./results', logging='INFO', wandb=False, downsample=True, as_test=False, continue_training=None, param_sharing=False)


In [33]:
# del trainer
trainer = "PPO"
default_config = ppo.PPOConfig()




def get_experiment_config(args, default_config):
    from meltingpot import substrate
    from ray.rllib.policy import policy
    if args.exp == 'pd_matrix':
        substrate_name = "prisoners_dilemma_in_the_matrix__repeated"
    elif args.exp == 'clean_up':
        substrate_name = "clean_up"
    else:
        raise Exception("Please set --exp to be one of ['pd_matrix', 'clean_up', \
                        ]. Other substrates are not supported.")

    # Fetch player roles
    player_roles = substrate.get_config(substrate_name).default_player_roles

    if args.downsample:
        scale_factor = 8
    else:
        scale_factor = 1

    params_dict = {

        # resources
        "num_rollout_workers": args.num_workers,
        "num_gpus": args.num_gpus,

        # Env
        "env_name": "meltingpot",
        "env_config": {"substrate": substrate_name, "roles": player_roles, "scaled": scale_factor},


        # training
        "seed": args.seed,
        # "rollout_fragment_length": 10,
        "train_batch_size": 8000,
        "sgd_minibatch_size": 1024,
        'num_sgd_iter': 20,
        "lr": 1e-4,
        "disable_observation_precprocessing": True,
        "use_new_rl_modules": False,
        "use_new_learner_api": False,
        "framework": args.framework,
        "exploration_config": {
            # "type":MyEpsExp,
            "type":"EpsilonGreedy",
            "warmup_timesteps": 1e5,
            "epsilon_timesteps": 1e7,
            
            },
        # "exploration_config":{
            # "type": "MyEpsExp",
            # "type":"RE3",
            # "sub_exploration":{
            #     "type": "StochasticSampling",
            # }
            # # "type":"EpsilonGreedy",
            # # "warmup_timesteps": 1e5,
            # # "epsilon_timesteps": 1e7,
            # },

        # agent model
        "fcnet_hidden": (256, 256),
        "post_fcnet_hidden": (128,),
        "cnn_activation": "silu",
        "fcnet_activation": "relu",
        "post_fcnet_activation": "relu",
        "use_lstm": True,
        "max_seq_len": 40,
        "lstm_use_prev_action": True,
        "lstm_use_prev_reward": False,
        "lstm_cell_size": 256,
        "shared_policy": args.param_sharing,

        # experiment trials
        "exp_name": args.exp,
        "stopping": {
                    # "timesteps_total": 1000000,
                    # "training_iteration": 100,
                    "episode_reward_mean": 100,
        },
        "num_checkpoints": 5,
        "checkpoint_interval": 50,
        "checkpoint_at_end": True,
        "results_dir": args.results_dir,
        "logging": args.logging,

    }

    
    # Preferrable to update the parameters in above dict before changing anything below
    
    run_configs: ppo.PPOConfig = default_config
    experiment_configs = {}
    tune_configs = None

    # Resources 
    run_configs.num_rollout_workers = params_dict['num_rollout_workers']
    run_configs.num_gpus = params_dict['num_gpus']


    # Training
    run_configs.train_batch_size = params_dict['train_batch_size']
    run_configs.sgd_minibatch_size = params_dict['sgd_minibatch_size']
    run_configs.num_sgd_iter = params_dict['num_sgd_iter']
    run_configs.lr = params_dict['lr']
    run_configs.preprocessor_pref = None
    run_configs._disable_preprocessor_api = params_dict['disable_observation_precprocessing']
    run_configs.rl_module(_enable_rl_module_api=params_dict['use_new_rl_modules'])
    run_configs.training(_enable_learner_api=params_dict['use_new_learner_api'])
    run_configs.exploration(exploration_config=params_dict['exploration_config'])
    run_configs = run_configs.framework(params_dict['framework'])
    run_configs.log_level = params_dict['logging']
    run_configs.seed = params_dict['seed']

    # Environment
    run_configs.env = params_dict['env_name']
    run_configs.env_config = params_dict['env_config']

    # Setup multi-agent policies. The below code will initialize independent
    # policies for each agent.
    if not params_dict['shared_policy']:
        base_env = make_envs.env_creator(run_configs.env_config)
        policies = {}
        player_to_agent = {}
        for i in range(len(player_roles)):
            rgb_shape = base_env.observation_space[f"player_{i}"]["RGB"].shape
            sprite_x = rgb_shape[0]
            sprite_y = rgb_shape[1]

            policies[f"agent_{i}"] = policy.PolicySpec(
                observation_space=base_env.observation_space[f"player_{i}"],
                action_space=base_env.action_space[f"player_{i}"],
                config={
                    "model": {
                        "conv_filters": [[16, [8, 8], 1],
                                        [128, [sprite_x, sprite_y], 1]],
                    },
                })
            player_to_agent[f"player_{i}"] = f"agent_{i}"

        run_configs.multi_agent(policies=policies, policy_mapping_fn=(lambda agent_id, *args, **kwargs: 
                                                                  player_to_agent[agent_id]))
    else:
        print("DEBUG: Use shared policy")
        base_env = make_envs.env_creator(run_configs.env_config)
        policies = {}
        rgb_shape = base_env.observation_space["player_0"]["RGB"].shape
        sprite_x = rgb_shape[0]
        sprite_y = rgb_shape[1]

        policies[f"shared_agent"] = policy.PolicySpec(
            observation_space=base_env.observation_space[f"player_0"],
            action_space=base_env.action_space[f"player_0"],
            config={
                "model": {
                    "conv_filters": [[16, [8, 8], 1],
                                    [128, [sprite_x, sprite_y], 1]],
                },
            })
        
        run_configs.multi_agent(policies=policies, policy_mapping_fn=(lambda agent_id, *args, **kwargs:
                                                                    "shared_agent"))
        
        
    
    run_configs.model["fcnet_hiddens"] = params_dict['fcnet_hidden']
    run_configs.model["post_fcnet_hiddens"] = params_dict['post_fcnet_hidden']
    run_configs.model["conv_activation"] = params_dict['cnn_activation'] 
    run_configs.model["fcnet_activation"] = params_dict['fcnet_activation']
    run_configs.model["post_fcnet_activation"] = params_dict['post_fcnet_activation']
    run_configs.model["use_lstm"] = params_dict['use_lstm']
    run_configs.model["max_seq_len"] = params_dict['max_seq_len']
    run_configs.model["lstm_use_prev_action"] = params_dict['lstm_use_prev_action']
    run_configs.model["lstm_use_prev_reward"] = params_dict['lstm_use_prev_reward']
    run_configs.model["lstm_cell_size"] = params_dict['lstm_cell_size']

    # Experiment Trials
    experiment_configs['name'] = params_dict['exp_name']
    experiment_configs['stop'] = params_dict['stopping']
    experiment_configs['keep_checkpoints_num'] = params_dict['num_checkpoints']
    experiment_configs['checkpoint_freq'] = params_dict['checkpoint_interval']
    experiment_configs['checkpoint_at_end'] = params_dict['checkpoint_at_end']
    # experiment_configs['keep'] = params_dict['num_checkpoints']
    # experiment_configs['freq'] = params_dict['checkpoint_interval']
    # experiment_configs['end'] = params_dict['checkpoint_at_end']
    if args.framework == 'tf':
        experiment_configs['local_dir'] = f"{params_dict['results_dir']}/tf"
    else:
        experiment_configs['local_dir'] = f"{params_dict['results_dir']}/torch"
 
    return run_configs, experiment_configs, tune_configs

configs, exp_config, _ = get_experiment_config(args, default_config)
# trainer:ppo.PPO = ppo.PPO.from_checkpoint(ckp)
# trainer.restore(ckp)
# print(f"Continuing training from {ckp}")

2024-05-24 15:53:33,584	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2024-05-24 15:53:33,586	WARNING algorithm_config.py:2548 -- Setting `exploration_config={'type': 'StochasticSampling'}` because you set `_enable_rl_module_api=False`. This exploration config was restored from a prior exploration config that was overriden when setting `_enable_rl_module_api=True`. This occurs because when RLModule API are enabled, exploration_config can not be set.


In [34]:
# trainer = ray.tune.registry.get_trainable_cls(trainer)
trainer_cls = ray.tune.tune._get_trainable(trainer)
trainer_obj = trainer_cls(configs, "meltingpot")

2024-05-24 15:53:37,058	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2024-05-24 15:53:37,059	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='meltingpot', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('meltingpot').build()` instead. This will raise an error in the future!
/nas/wjxie/anaconda3/envs/gtp/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be re

In [24]:
print(trainer_obj)
# print(dir(trainer_obj))
# print(trainer_obj.__dict__)

PPO


In [36]:
# trainer.training_iteration()
worker:ray.rllib.evaluation.rollout_worker.RolloutWorker= trainer_obj.workers.local_worker()
policy_spec = worker.policy_dict['agent_0']
policy_instance = worker.policy_map['agent_0']
nn_model:nn.Module = policy_instance.model 
# print(dir(worker))
# print(isinstance(nn_model,nn.Module))
print("Total_params: ", sum([param.nelement() for param in nn_model.parameters()]))
# print(nn_model.state_dict().keys())
print(nn_model._modules.keys())
print("-"*100)
print(nn_model)
# print("Total_params: ", nn_model.total_parameters())

Total_params:  1143598
odict_keys(['cnns', 'one_hot', 'flatten', 'flatten_0', 'flatten_1', 'flatten_2', 'cnn_3', 'post_fc_stack', 'lstm', '_logits_branch', '_value_branch'])
----------------------------------------------------------------------------------------------------
ComplexInputNetwork_as_LSTMWrapper(
  (cnns): ModuleDict(
    (3): VisionNetwork(
      (_convs): Sequential(
        (0): SlimConv2d(
          (_model): Sequential(
            (0): ZeroPad2d((3, 4, 3, 4))
            (1): Conv2d(3, 16, kernel_size=(8, 8), stride=(1, 1))
            (2): SiLU()
          )
        )
        (1): SlimConv2d(
          (_model): Sequential(
            (0): Conv2d(16, 128, kernel_size=(5, 5), stride=(1, 1))
            (1): SiLU()
          )
        )
        (2): Flatten(start_dim=1, end_dim=-1)
      )
      (_value_branch_separate): Sequential(
        (0): SlimConv2d(
          (_model): Sequential(
            (0): ZeroPad2d((3, 4, 3, 4))
            (1): Conv2d(3, 16, kernel_